In [1]:
import numpy as np
import pickle
import pandas as pd
import glob
import os

## Step 1: make probe fasta

Load the probe dataset

The .ndf file describes the design of the Roche Nimblegen microarray chips that were used in this study. This file is renamed from the GPL6759.ndf file corresponding to the raw data repository GEO accession GSE11300, which can be downloaded from https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE11300&format=file

In [2]:
file_path = './raw_data/GSE276404/GPL6759.ndf.gz'

probe_df = pd.read_csv(file_path, sep='\t', compression='gzip')

Take a look

In [3]:
probe_df.head()

,PROBE_DESIGN_ID,CONTAINER,DESIGN_NOTE,SELECTION_CRITERIA,SEQ_ID,PROBE_SEQUENCE,MISMATCH,MATCH_INDEX,FEATURE_ID,ROW_NUM,COL_NUM,PROBE_CLASS,PROBE_ID,POSITION,DESIGN_ID,X,Y
0,5314_0001_0001,BLOCK1,rank_selected,rank:09;score:376;uniq:03;count:37;freq:00;rul...,TETRA00S0021925,AGATATGTATGACTCTATACATAGAAATGTGAATGATATAGTTAGC...,0,64456195,64456195,1,1,experimental,TETRAP00318583,2262,5314,1,1
1,5314_0023_0001,BLOCK1,rank_selected,rank:09;score:402;uniq:26;count:37;freq:00;rul...,TETRA00S0012676,AAGGGCGCATTAGTGCCAAATGGGACACGTAGAAAGGTAGAGGGAT...,0,64330340,64330340,1,23,experimental,TETRAP00183246,3282,5314,23,1
2,5314_0025_0001,BLOCK1,rank_selected,rank:08;score:115;uniq:01;count:31;freq:00;rul...,TETRA00S0002513,TATCATATAAAAATGTGTCTTGTTTATAAGACATTAATTGCTTAAT...,0,64193974,64193974,1,25,experimental,TETRAP00036232,262,5314,25,1
3,5314_0027_0001,BLOCK1,rank_selected,rank:11;score:341;uniq:23;count:37;freq:00;rul...,TETRA00S0000062,AGATTTGATGTAATATAAGTTATCCGAAACATGGCAGTAGAATGCA...,0,64161203,64161203,1,27,experimental,TETRAP00000895,825,5314,27,1
4,5314_0029_0001,BLOCK1,rank_selected,rank:01;score:560;uniq:21;count:37;freq:00;rul...,TETRA00S0006635,TAAATAACTAAGAGCATAAGGCTGGTGCAAGTAACTCAATGCAACC...,0,64249458,64249458,1,29,experimental,TETRAP00096103,3290,5314,29,1


Note: the SEQ_ID is what the chip designers thought the genes were (later converted to TTHERM_ format); the PROBE_ID is unique for each PROBE_SEQUENCE. Consequently, for all of the quality control and filtering, we want to work with the PROBE_IDs, which we can then map to our current understanding of the genome by their sequence.

The seq ids are not unique because multiple probes can target a single gene. Keep in mind that our understanding of what genes exist has dramatically changed since these microarrays were designed.

In [4]:
len(probe_df)

392778

This length is indicative of a 1:2 design for the chip. See http://mtweb.cs.ucl.ac.uk/mus/mus/binnaz/CNV/NimbleGene/DATA/OID8421-2/Documentation/NimbleGen_data_formats.pdf for more info

In [5]:
probe_df.loc[probe_df['SEQ_ID'] == 'TETRA00S0000001']

,PROBE_DESIGN_ID,CONTAINER,DESIGN_NOTE,SELECTION_CRITERIA,SEQ_ID,PROBE_SEQUENCE,MISMATCH,MATCH_INDEX,FEATURE_ID,ROW_NUM,COL_NUM,PROBE_CLASS,PROBE_ID,POSITION,DESIGN_ID,X,Y
6400,5314_0222_0018,BLOCK1,rank_selected,rank:04;score:424;uniq:15;count:37;freq:00;rul...,TETRA00S0000001,CATACAATCTTCTAATTACTTGCACTAGAGATTTTCAACGTATTGG...,0,64160384,64160384,18,222,experimental,TETRAP00000010,791,5314,222,18
74576,5314_0008_0196,BLOCK1,rank_selected,rank:05;score:387;uniq:15;count:37;freq:00;rul...,TETRA00S0000001,ATAGATTAAAGCTCTGATGACTCTCCTTCGCCTTAAATATAATCAT...,0,64160388,64160388,196,8,experimental,TETRAP00000015,1230,5314,8,196
132820,5314_0527_0347,BLOCK1,rank_selected,rank:07;score:359;uniq:05;count:37;freq:00;rul...,TETRA00S0000001,TTTTTCAATGAAGTGAAGATGCTTAGAACATTGAACCACAAGCTAA...,0,64160379,64160379,347,527,experimental,TETRAP00000005,517,5314,527,347
137513,5314_0697_0359,BLOCK1,rank_selected,rank:02;score:468;uniq:14;count:37;freq:00;rul...,TETRA00S0000001,CTTAATGATAAGATAACCTACTAAATGATAATTGATGACGAGACGA...,0,64160376,64160376,359,697,experimental,TETRAP00000002,211,5314,697,359
145827,5314_0429_0381,BLOCK1,rank_selected,rank:01;score:482;uniq:15;count:37;freq:00;rul...,TETRA00S0000001,CTTTTATAGGCTGGTTGCAGGAAAGACATATCAATAATCCTTGACA...,0,64160386,64160386,381,429,experimental,TETRAP00000012,954,5314,429,381
151844,5314_0175_0397,BLOCK1,rank_selected,rank:03;score:434;uniq:13;count:37;freq:00;rul...,TETRA00S0000001,TTAACATTGTAAGAAGTCTGCGTGATAATAGAGCAAATATGCTAAG...,0,64160381,64160381,397,175,experimental,TETRAP00000007,667,5314,175,397
178349,5314_0194_0466,BLOCK1,rank_selected,rank:12;score:327;uniq:12;count:37;freq:00;rul...,TETRA00S0000001,ACAAAGAGTTTAACATTGTAAGAAGTCTGCGTGATAATAGAGCAAA...,0,64160380,64160380,466,194,experimental,TETRAP00000006,658,5314,194,466
178366,5314_0228_0466,BLOCK1,rank_selected,rank:11;score:333;uniq:12;count:37;freq:00;rul...,TETRA00S0000001,GTATAACAACTCATACAATCTTCTAATTACTTGCACTAGAGATTTT...,0,64160382,64160382,466,228,experimental,TETRAP00000008,780,5314,228,466
191342,5314_0130_0500,BLOCK1,rank_selected,rank:06;score:382;uniq:20;count:34;freq:00;rul...,TETRA00S0000001,GAGTTTACACGGTTCAGGACACCTCAGGCTGCATGAAAATAAAATA...,0,64160375,64160375,500,130,experimental,TETRAP00000001,126,5314,130,500
209910,5314_0011_0549,BLOCK1,rank_selected,rank:08;score:355;uniq:05;count:37;freq:00;rul...,TETRA00S0000001,AAGTAAATGCTTGATAATAACCCTGATAAGAGACCTTCTGCAGATG...,0,64160387,64160387,549,11,experimental,TETRAP00000014,1084,5314,11,549


In [6]:
len(probe_df['SEQ_ID'].values) == len(probe_df['SEQ_ID'].unique())

False

Can find negative controls if need be

In [7]:
probe_df['DESIGN_NOTE'].unique()

array(['rank_selected', nan, '-', 'upper left fiducial', '0 cycles',
       '04 cycles', '08 cycles', '12 cycles', '16 cycles', '20 cycles',
       '24 cycles', '28 cycles', '32 cycles', '36 cycles', '40 cycles',
       '44 cycles', '48 cycles', '52 cycles', '56 cycles', '60 cycles',
       '64 cycles', '68 cycles', '72 cycles', '76 cycles', '80 cycles',
       '84 cycles', '88 cycles', '92 cycles', '96 cycles', '100 cycles',
       '104 cycles', 'synthesis control', 'upper center fiducial',
       'upper right fiducial', 'REPLICATE1', 'EMPTY',
       'upper right chip_id', '+', 'uniformity control',
       'vertical design_id', 'left center fiducial',
       'center cross fiducial', 'right center fiducial',
       'lower center fiducial', 'REPLICATE2', 'lower left fiducial',
       'lower right fiducial', 'lower left chip_id',
       'horizontal design_id'], dtype=object)

In [8]:
probe_df.loc[probe_df['DESIGN_NOTE'] == '-']

,PROBE_DESIGN_ID,CONTAINER,DESIGN_NOTE,SELECTION_CRITERIA,SEQ_ID,PROBE_SEQUENCE,MISMATCH,MATCH_INDEX,FEATURE_ID,ROW_NUM,COL_NUM,PROBE_CLASS,PROBE_ID,POSITION,DESIGN_ID,X,Y
158,5314_0491_0001,NGS_CONTROLS,-,T/A,ARRAY_QC_A,ACGTCCCCCTCTGGaTGTTCATACGGTATG,10004,1100011,62205230,1,491,control:reseq_qc:synthesis,XENOSYNTH0093,12,5314,491,1
523,5314_0492_0002,NGS_CONTROLS,-,T/C,ARRAY_QC_A,ACGTCCCCCTCTGGcTGTTCATACGGTATG,10005,1100011,62205231,2,492,control:reseq_qc:synthesis,XENOSYNTH0094,12,5314,492,2
818,5314_0491_0003,NGS_CONTROLS,-,T/G,ARRAY_QC_A,ACGTCCCCCTCTGGgTGTTCATACGGTATG,10006,1100011,62205232,3,491,control:reseq_qc:synthesis,XENOSYNTH0095,12,5314,491,3
1190,5314_0492_0004,NGS_CONTROLS,-,T/T,ARRAY_QC_A,ACGTCCCCCTCTGGtTGTTCATACGGTATG,10007,1100011,62205233,4,492,control:reseq_qc:synthesis,XENOSYNTH0096,12,5314,492,4
6079,5314_0341_0017,NGS_CONTROLS,-,G/A,ARRAY_QC_C,GCGCGGCGTTGGACaTCTGACTAATACATCAA,10004,1100089,62205854,17,341,control:reseq_qc:synthesis,XENOSYNTH0717,90,5314,341,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389799,5314_0434_1016,NGS_CONTROLS,-,G/T,ARRAY_QC_C,ATGTGCGCGGCGTTtGACGTCTGACTAAT,10007,1100093,62205889,1016,434,control:reseq_qc:synthesis,XENOSYNTH0752,94,5314,434,1016
390294,5314_0661_1017,NGS_CONTROLS,-,A/A,LABEL_QC_B,ATTAGGCCCTTCGCaCGCAGCGGCGTGCG,10004,1100155,62206382,1017,661,control:reseq_qc:label,XENOLABEL0445,56,5314,661,1017
390675,5314_0662_1018,NGS_CONTROLS,-,A/C,LABEL_QC_B,ATTAGGCCCTTCGCcCGCAGCGGCGTGCG,10005,1100155,62206383,1018,662,control:reseq_qc:label,XENOLABEL0446,56,5314,662,1018
391056,5314_0661_1019,NGS_CONTROLS,-,A/G,LABEL_QC_B,ATTAGGCCCTTCGCgCGCAGCGGCGTGCG,10006,1100155,62206384,1019,661,control:reseq_qc:label,XENOLABEL0447,56,5314,661,1019


Example of many probes to a single id

In [9]:
probe_df.loc[probe_df['SEQ_ID']=='TETRA00S0021925']

,PROBE_DESIGN_ID,CONTAINER,DESIGN_NOTE,SELECTION_CRITERIA,SEQ_ID,PROBE_SEQUENCE,MISMATCH,MATCH_INDEX,FEATURE_ID,ROW_NUM,COL_NUM,PROBE_CLASS,PROBE_ID,POSITION,DESIGN_ID,X,Y
0,5314_0001_0001,BLOCK1,rank_selected,rank:09;score:376;uniq:03;count:37;freq:00;rul...,TETRA00S0021925,AGATATGTATGACTCTATACATAGAAATGTGAATGATATAGTTAGC...,0,64456195,64456195,1,1,experimental,TETRAP00318583,2262,5314,1,1
12641,5314_0474_0034,BLOCK1,rank_selected,rank:14;score:346;uniq:13;count:37;freq:00;rul...,TETRA00S0021925,TAATAATTTAATAGCGGATAGTCGATAATGTCAAACAGCATTTAAA...,0,64456191,64456191,34,474,experimental,TETRAP00318579,850,5314,474,34
67117,5314_0450_0176,BLOCK1,rank_selected,rank:08;score:378;uniq:05;count:37;freq:00;rul...,TETRA00S0021925,TCAAATATGTACCCCATTACATATCAAACAATGATGAGTTAAAACC...,0,64456194,64456194,176,450,experimental,TETRAP00318582,1777,5314,450,176
82746,5314_0219_0217,BLOCK1,rank_selected,rank:12;score:353;uniq:00;count:37;freq:00;rul...,TETRA00S0021925,CTGTCAGCTCAATCTTCTTACTTTCTGATGGTCAGGACAATAATTC...,0,64456200,64456200,217,219,experimental,TETRAP00318588,5102,5314,219,217
128974,5314_0515_0337,BLOCK1,rank_selected,rank:05;score:423;uniq:06;count:37;freq:00;rul...,TETRA00S0021925,AATTTAGCAATATGAAATCAATAACTAGGCCAAGATATATGCACAA...,0,64456196,64456196,337,515,experimental,TETRAP00318584,2706,5314,515,337
149412,5314_0688_0390,BLOCK1,rank_selected,rank:02;score:487;uniq:09;count:37;freq:00;rul...,TETRA00S0021925,TCGTTTTAGAAATATAGATAGGGTGGAGTTACCAATAGATTGATTA...,0,64456190,64456190,390,688,experimental,TETRAP00318578,742,5314,688,390
203654,5314_0520_0532,BLOCK1,rank_selected,rank:13;score:349;uniq:00;count:37;freq:00;rul...,TETRA00S0021925,TTGATTAAAATGGCAATGATGCTTTCAATCAAAGTGATGATTTAGC...,0,64456192,64456192,532,520,experimental,TETRAP00318580,1133,5314,520,532
208163,5314_0358_0544,BLOCK1,rank_selected,rank:04;score:433;uniq:09;count:37;freq:00;rul...,TETRA00S0021925,GCAATTCATAACATAATTAGATTTAATTCTATCCATAGCCAACATC...,0,64456193,64456193,544,358,experimental,TETRAP00318581,1487,5314,358,544
208452,5314_0167_0545,BLOCK1,rank_selected,rank:03;score:466;uniq:10;count:37;freq:00;rul...,TETRA00S0021925,GAATAGCATAATAATGGATGGCATTATCATAGAGAGTATTGACAAG...,0,64456198,64456198,545,167,experimental,TETRAP00318586,3792,5314,167,545
221084,5314_0088_0578,BLOCK1,rank_selected,rank:11;score:353;uniq:08;count:37;freq:00;rul...,TETRA00S0021925,TGAATATATAGATGGTTAAAACAATATAATTTACGACTCGAATGAG...,0,64456197,64456197,578,88,experimental,TETRAP00318585,3168,5314,88,578


The PROBE_IDs are also not unique

In [10]:
len(probe_df['PROBE_ID'].values) == len(probe_df['PROBE_ID'].unique())

False

In [11]:
probe_df[probe_df.duplicated(subset=['PROBE_ID'])]

,PROBE_DESIGN_ID,CONTAINER,DESIGN_NOTE,SELECTION_CRITERIA,SEQ_ID,PROBE_SEQUENCE,MISMATCH,MATCH_INDEX,FEATURE_ID,ROW_NUM,COL_NUM,PROBE_CLASS,PROBE_ID,POSITION,DESIGN_ID,X,Y
284,5314_0005_0002,NGS_CONTROLS,upper left fiducial,bright,FIDUCIAL_UPPER_LEFT,TGAGTTGTTTGATAGGATTATTCATAGAGGTCATTACAGCGAGAGG...,0,2000197,62205062,2,5,fiducial,CPK6,0,5314,5,2
285,5314_0007_0002,NGS_CONTROLS,upper left fiducial,bright,FIDUCIAL_UPPER_LEFT,TGAGTTGTTTGATAGGATTATTCATAGAGGTCATTACAGCGAGAGG...,0,2000198,62205063,2,7,fiducial,CPK6,0,5314,7,2
286,5314_0009_0002,NGS_CONTROLS,upper left fiducial,bright,FIDUCIAL_UPPER_LEFT,TGAGTTGTTTGATAGGATTATTCATAGAGGTCATTACAGCGAGAGG...,0,2000199,62205064,2,9,fiducial,CPK6,0,5314,9,2
287,5314_0011_0002,NGS_CONTROLS,upper left fiducial,bright,FIDUCIAL_UPPER_LEFT,TGAGTTGTTTGATAGGATTATTCATAGAGGTCATTACAGCGAGAGG...,0,2000200,62205065,2,11,fiducial,CPK6,0,5314,11,2
288,5314_0013_0002,NGS_CONTROLS,upper left fiducial,bright,FIDUCIAL_UPPER_LEFT,TGAGTTGTTTGATAGGATTATTCATAGAGGTCATTACAGCGAGAGG...,0,2000201,62205066,2,13,fiducial,CPK6,0,5314,13,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392507,5314_0118_1024,H_CODE,horizontal design_id,dark,DESIGN_ID1,N,0,1060094,63771045,3,31,encoded number,empty,15,5314,118,1024
392508,5314_0120_1024,H_CODE,horizontal design_id,dark,DESIGN_ID1,N,0,1060100,63771051,3,33,encoded number,empty,16,5314,120,1024
392509,5314_0122_1024,H_CODE,horizontal design_id,dark,DESIGN_ID1,N,0,1060106,63771057,3,35,encoded number,empty,17,5314,122,1024
392510,5314_0124_1024,H_CODE,horizontal design_id,dark,DESIGN_ID1,N,0,1060112,63771063,3,37,encoded number,empty,18,5314,124,1024


In [12]:
probe_df['PROBE_CLASS'].unique()

array(['experimental', nan, 'control:reseq_qc:synthesis', 'fiducial',
       'linker', 'synthesis', 'control:sample_tracking:A',
       'control:empty', 'encoded number', 'control:reseq_qc:label',
       'uniformity', 'control', 'control:sample_tracking:B'], dtype=object)

In [13]:
probe_df[probe_df.duplicated(subset=['PROBE_ID'])]['PROBE_CLASS'].unique()

array(['fiducial', 'linker', 'synthesis', 'control:empty',
       'encoded number', 'uniformity', 'control',
       'control:sample_tracking:B'], dtype=object)

These are all controls of various sorts, etc. and I can exclude them.

In [14]:
experimental_probe_df = probe_df.loc[probe_df['PROBE_CLASS']=='experimental']

In [15]:
experimental_probe_df.head()

,PROBE_DESIGN_ID,CONTAINER,DESIGN_NOTE,SELECTION_CRITERIA,SEQ_ID,PROBE_SEQUENCE,MISMATCH,MATCH_INDEX,FEATURE_ID,ROW_NUM,COL_NUM,PROBE_CLASS,PROBE_ID,POSITION,DESIGN_ID,X,Y
0,5314_0001_0001,BLOCK1,rank_selected,rank:09;score:376;uniq:03;count:37;freq:00;rul...,TETRA00S0021925,AGATATGTATGACTCTATACATAGAAATGTGAATGATATAGTTAGC...,0,64456195,64456195,1,1,experimental,TETRAP00318583,2262,5314,1,1
1,5314_0023_0001,BLOCK1,rank_selected,rank:09;score:402;uniq:26;count:37;freq:00;rul...,TETRA00S0012676,AAGGGCGCATTAGTGCCAAATGGGACACGTAGAAAGGTAGAGGGAT...,0,64330340,64330340,1,23,experimental,TETRAP00183246,3282,5314,23,1
2,5314_0025_0001,BLOCK1,rank_selected,rank:08;score:115;uniq:01;count:31;freq:00;rul...,TETRA00S0002513,TATCATATAAAAATGTGTCTTGTTTATAAGACATTAATTGCTTAAT...,0,64193974,64193974,1,25,experimental,TETRAP00036232,262,5314,25,1
3,5314_0027_0001,BLOCK1,rank_selected,rank:11;score:341;uniq:23;count:37;freq:00;rul...,TETRA00S0000062,AGATTTGATGTAATATAAGTTATCCGAAACATGGCAGTAGAATGCA...,0,64161203,64161203,1,27,experimental,TETRAP00000895,825,5314,27,1
4,5314_0029_0001,BLOCK1,rank_selected,rank:01;score:560;uniq:21;count:37;freq:00;rul...,TETRA00S0006635,TAAATAACTAAGAGCATAAGGCTGGTGCAAGTAACTCAATGCAACC...,0,64249458,64249458,1,29,experimental,TETRAP00096103,3290,5314,29,1


In [16]:
len(probe_df)

392778

In [17]:
len(experimental_probe_df)

384999

In [18]:
experimental_probe_df['MISMATCH'].unique()

array([0])

In [19]:
with open('./microarray_probe_alignment.sam', 'r') as f:
    lines = f.readlines()
    single_alignments = [line for line in lines if line.split()[-1] == 'NH:i:1']

Sanity check that there are fewer single alignments than total alignments

In [20]:
len(lines)

403143

In [21]:
lines[0]

'TETRAP00318583\t0\tTTHERM_00709600\t807\t60\t60M\t*\t0\t0\tAGATATGTATGACTCTATACATAGAAATGTGAATGATATAGTTAGCAAATATTTTAAGCC\tIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII\tAS:i:0\tXN:i:0\tXM:i:0\tXO:i:0\tXG:i:0\tNM:i:0\tMD:Z:60\tYT:Z:UU\tNH:i:1\n'

In [22]:
len(single_alignments)

313654

In [23]:
single_alignments[0].split()

['TETRAP00318583',
 '0',
 'TTHERM_00709600',
 '807',
 '60',
 '60M',
 '*',
 '0',
 '0',
 'AGATATGTATGACTCTATACATAGAAATGTGAATGATATAGTTAGCAAATATTTTAAGCC',
 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII',
 'AS:i:0',
 'XN:i:0',
 'XM:i:0',
 'XO:i:0',
 'XG:i:0',
 'NM:i:0',
 'MD:Z:60',
 'YT:Z:UU',
 'NH:i:1']

Take a look at the formatting

In [24]:
single_alignments[0].split()

['TETRAP00318583',
 '0',
 'TTHERM_00709600',
 '807',
 '60',
 '60M',
 '*',
 '0',
 '0',
 'AGATATGTATGACTCTATACATAGAAATGTGAATGATATAGTTAGCAAATATTTTAAGCC',
 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII',
 'AS:i:0',
 'XN:i:0',
 'XM:i:0',
 'XO:i:0',
 'XG:i:0',
 'NM:i:0',
 'MD:Z:60',
 'YT:Z:UU',
 'NH:i:1']

In [25]:
# unique mapping frequencies
np.unique([sa.split()[-1] for sa in lines])

array(['NH:i:1', 'NH:i:2', 'NH:i:3', 'NH:i:4', 'NH:i:5', 'YT:Z:UU'],
      dtype='<U7')

Build probe_id to ttherm_id dictionary

In [26]:
single_aligned_probes = [line.split()[0] for line in single_alignments]

In [27]:
single_aligned_probes[:20]

['TETRAP00318583',
 'TETRAP00183246',
 'TETRAP00000895',
 'TETRAP00096103',
 'TETRAP00268899',
 'TETRAP00314657',
 'TETRAP00037459',
 'TETRAP00082274',
 'TETRAP00315441',
 'TETRAP00155304',
 'TETRAP00151274',
 'TETRAP00232583',
 'TETRAP00358039',
 'TETRAP00189264',
 'TETRAP00075363',
 'TETRAP00295958',
 'TETRAP00121342',
 'TETRAP00315221',
 'TETRAP00025855',
 'TETRAP00149569']

In [28]:
with open('./single_aligned_probes2024.pkl', 'wb') as f:
    pickle.dump(single_aligned_probes, f)

In [29]:
align_dict = {}
for alignment in single_alignments:
    split_alignment = alignment.split()
    align_dict[split_alignment[0]] = split_alignment[2]

In [30]:
align_dict['TETRAP00177701']

'TTHERM_00486275'

# BUILD NDF

In [31]:
filtered_probe_df = probe_df.loc[(probe_df['CONTAINER'] == 'RANDOM') | (probe_df['PROBE_CLASS'] == 'experimental')].copy()
filtered_probe_df.head()

,PROBE_DESIGN_ID,CONTAINER,DESIGN_NOTE,SELECTION_CRITERIA,SEQ_ID,PROBE_SEQUENCE,MISMATCH,MATCH_INDEX,FEATURE_ID,ROW_NUM,COL_NUM,PROBE_CLASS,PROBE_ID,POSITION,DESIGN_ID,X,Y
0,5314_0001_0001,BLOCK1,rank_selected,rank:09;score:376;uniq:03;count:37;freq:00;rul...,TETRA00S0021925,AGATATGTATGACTCTATACATAGAAATGTGAATGATATAGTTAGC...,0,64456195,64456195,1,1,experimental,TETRAP00318583,2262,5314,1,1
1,5314_0023_0001,BLOCK1,rank_selected,rank:09;score:402;uniq:26;count:37;freq:00;rul...,TETRA00S0012676,AAGGGCGCATTAGTGCCAAATGGGACACGTAGAAAGGTAGAGGGAT...,0,64330340,64330340,1,23,experimental,TETRAP00183246,3282,5314,23,1
2,5314_0025_0001,BLOCK1,rank_selected,rank:08;score:115;uniq:01;count:31;freq:00;rul...,TETRA00S0002513,TATCATATAAAAATGTGTCTTGTTTATAAGACATTAATTGCTTAAT...,0,64193974,64193974,1,25,experimental,TETRAP00036232,262,5314,25,1
3,5314_0027_0001,BLOCK1,rank_selected,rank:11;score:341;uniq:23;count:37;freq:00;rul...,TETRA00S0000062,AGATTTGATGTAATATAAGTTATCCGAAACATGGCAGTAGAATGCA...,0,64161203,64161203,1,27,experimental,TETRAP00000895,825,5314,27,1
4,5314_0029_0001,BLOCK1,rank_selected,rank:01;score:560;uniq:21;count:37;freq:00;rul...,TETRA00S0006635,TAAATAACTAAGAGCATAAGGCTGGTGCAAGTAACTCAATGCAACC...,0,64249458,64249458,1,29,experimental,TETRAP00096103,3290,5314,29,1


In [32]:
filtered_probe_df['SEQ_ID'] = [align_dict[probe_id] if probe_id in align_dict else 'NA' for probe_id in filtered_probe_df['PROBE_ID'].values]
filtered_probe_df.head()

,PROBE_DESIGN_ID,CONTAINER,DESIGN_NOTE,SELECTION_CRITERIA,SEQ_ID,PROBE_SEQUENCE,MISMATCH,MATCH_INDEX,FEATURE_ID,ROW_NUM,COL_NUM,PROBE_CLASS,PROBE_ID,POSITION,DESIGN_ID,X,Y
0,5314_0001_0001,BLOCK1,rank_selected,rank:09;score:376;uniq:03;count:37;freq:00;rul...,TTHERM_00709600,AGATATGTATGACTCTATACATAGAAATGTGAATGATATAGTTAGC...,0,64456195,64456195,1,1,experimental,TETRAP00318583,2262,5314,1,1
1,5314_0023_0001,BLOCK1,rank_selected,rank:09;score:402;uniq:26;count:37;freq:00;rul...,TTHERM_00529480,AAGGGCGCATTAGTGCCAAATGGGACACGTAGAAAGGTAGAGGGAT...,0,64330340,64330340,1,23,experimental,TETRAP00183246,3282,5314,23,1
2,5314_0025_0001,BLOCK1,rank_selected,rank:08;score:115;uniq:01;count:31;freq:00;rul...,NA,TATCATATAAAAATGTGTCTTGTTTATAAGACATTAATTGCTTAAT...,0,64193974,64193974,1,25,experimental,TETRAP00036232,262,5314,25,1
3,5314_0027_0001,BLOCK1,rank_selected,rank:11;score:341;uniq:23;count:37;freq:00;rul...,TTHERM_00002620,AGATTTGATGTAATATAAGTTATCCGAAACATGGCAGTAGAATGCA...,0,64161203,64161203,1,27,experimental,TETRAP00000895,825,5314,27,1
4,5314_0029_0001,BLOCK1,rank_selected,rank:01;score:560;uniq:21;count:37;freq:00;rul...,TTHERM_01013320,TAAATAACTAAGAGCATAAGGCTGGTGCAAGTAACTCAATGCAACC...,0,64249458,64249458,1,29,experimental,TETRAP00096103,3290,5314,29,1


In [33]:
df_single_alignments_filtered_probe = filtered_probe_df.loc[filtered_probe_df['SEQ_ID'] != 'NA']
df_single_alignments_filtered_probe.head()

,PROBE_DESIGN_ID,CONTAINER,DESIGN_NOTE,SELECTION_CRITERIA,SEQ_ID,PROBE_SEQUENCE,MISMATCH,MATCH_INDEX,FEATURE_ID,ROW_NUM,COL_NUM,PROBE_CLASS,PROBE_ID,POSITION,DESIGN_ID,X,Y
0,5314_0001_0001,BLOCK1,rank_selected,rank:09;score:376;uniq:03;count:37;freq:00;rul...,TTHERM_00709600,AGATATGTATGACTCTATACATAGAAATGTGAATGATATAGTTAGC...,0,64456195,64456195,1,1,experimental,TETRAP00318583,2262,5314,1,1
1,5314_0023_0001,BLOCK1,rank_selected,rank:09;score:402;uniq:26;count:37;freq:00;rul...,TTHERM_00529480,AAGGGCGCATTAGTGCCAAATGGGACACGTAGAAAGGTAGAGGGAT...,0,64330340,64330340,1,23,experimental,TETRAP00183246,3282,5314,23,1
3,5314_0027_0001,BLOCK1,rank_selected,rank:11;score:341;uniq:23;count:37;freq:00;rul...,TTHERM_00002620,AGATTTGATGTAATATAAGTTATCCGAAACATGGCAGTAGAATGCA...,0,64161203,64161203,1,27,experimental,TETRAP00000895,825,5314,27,1
4,5314_0029_0001,BLOCK1,rank_selected,rank:01;score:560;uniq:21;count:37;freq:00;rul...,TTHERM_01013320,TAAATAACTAAGAGCATAAGGCTGGTGCAAGTAACTCAATGCAACC...,0,64249458,64249458,1,29,experimental,TETRAP00096103,3290,5314,29,1
5,5314_0031_0001,BLOCK1,rank_selected,rank:04;score:459;uniq:13;count:37;freq:00;rul...,TTHERM_00455220,CTTTTTCAATCCTACTGCAATCTTGAAGCCCTGTATAATAGATAAT...,0,64409997,64409997,1,31,experimental,TETRAP00268899,754,5314,31,1


In [34]:
df_single_alignments_filtered_probe.shape

(313654, 17)

In [35]:
len(df_single_alignments_filtered_probe['SEQ_ID'].unique())

23660

In [36]:
output_file_path = './raw_data/filtered_tetexpr.ndf'

df_single_alignments_filtered_probe.to_csv(output_file_path, sep='\t', index=False)

In [40]:
def pair_to_xys(filtered_probe_df, path_to_pair, xys_filename):
    
    """
    Function to convert .pair format into .xys format
    """

    if '1772502_532' in path_to_pair:
        cols = ['IMAGE_ID', 'GENE_EXPR_OPTION', 'SEQ_ID', 'PROBE_ID', 'POSITION', 'X',
       'Y', 'MATCH_INDEX', 'SEQ_URL', 'PM', 'MM']
        pair = pd.read_csv(path_to_pair, comment='#', sep='\t', compression='gzip', header=None)
        pair.columns = cols
    else:
        pair = pd.read_csv(path_to_pair, comment='#', sep='\t', compression='gzip')

    probe_signal_df = pair[['PROBE_ID', 'PM']]
    probe_signal_df = probe_signal_df.rename(columns={'PM': 'SIGNAL'})
    
    m = filtered_probe_df.merge(probe_signal_df, on='PROBE_ID') # FIXME random probes named differently
    
#     counts = [1 if p == 'experimental' or c == 'RANDOM' else 'NA' for p, c in zip(m['PROBE_CLASS'].values, m['CONTAINER'].values)]
    
    m['COUNT'] = [1 for _ in range(m.shape[0])]
    
#     m = m.dropna()
    
    xys = m[['X', 'Y', 'SIGNAL', 'COUNT']]
    
#     xys = m[['PROBE_ID', 'X', 'Y', 'SIGNAL', 'COUNT']]

    
    xys.to_csv(xys_filename, sep='\t', index=False)
    
    with open(xys_filename, 'r+') as f:
        
        content = f.read()
        f.seek(0, 0)
        f.write('# designname=2021_tetrahymena_expr_corrected date=2021_11_07' + '\n' + content)

In [41]:
raw_microarray_data_files = glob.glob('./raw_data/expression/*.gz')

In [42]:
for f in raw_microarray_data_files:
    file_basename = os.path.basename(f)
    file_name = f"./raw_data/expression/{'_'.join(file_basename.split('_')[1:3])}.xys"
    pair_to_xys(filtered_probe_df, f, file_name)